# Introduction

This tutorial demonstrates how to calibrate real and reciprocal space coordinates of scanning electron diffraction data, including correcting the diffraction pattern for lens distortions, using data acquired an Au X-grating reference standard.

Lens distortions are assumed to be dominated by the projector lens system and it is assumed that that elliptical distortion of the diffraction pattern tends to dominate over other lens distortions such as pincushion, barrel and spiral distortions.

Distortion correction is based on measuring the ellipticity of a ring pattern obtained from an Au X-grating calibration standard in scaninng mode.

All functionality has been checked using pyxem-0.8.0

# Contents

1. <a href='#ini'> Load Data & Initialize Generator</a>
2. <a href='#dis'> Determine Lens Distortions</a>
3. <a href='#cal'> Determine Real & Reciprocal Space Calibrations</a>

Import pyxem, required libraries and pyxem modules

In [ ]:
%matplotlib tk
import numpy as np
import pyxem as pxm

from pyxem.libraries.calibration_library import CalibrationDataLibrary
from pyxem.generators.calibration_generator import CalibrationGenerator

# <a id='ini'></a> 1. Load Data & Initialize Generator

Load a ring pattern calculated as the mean diffraction from SED data acquired from an Au X-grating after centering the data

In [ ]:
dpeg = pxm.load('./au_xgrating_20cm.tif')
dpeg = pxm.ElectronDiffraction(dpeg)

Load a VDF image of the Au X-grating

In [ ]:
im = pxm.load('./au_xgrating_100kX.hspy')

Define a CalibrationDataLibary as a container for calibration data.

In [ ]:
cal_lib = CalibrationDataLibrary(au_x_grating_dp=dpeg,
                                 au_x_grating_im=im)

Plot the calibration library data for inspection

In [ ]:
cal_lib.plot_calibration_data(data_to_plot='au_x_grating_dp',
                              cmap='inferno', vmax=0.1)

In [ ]:
cal_lib.plot_calibration_data(data_to_plot='au_x_grating_im')

Initialise a CalibrationGenerator with the CalibrationDataLibrary

In [ ]:
cal = CalibrationGenerator(calibration_data=cal_lib)

# <a id='ids'></a> 2. Determine Lens Distortions

Determine distortion correction matrix by ring fitting

In [ ]:
cal.get_elliptical_distortion(mask_radius=10,
                              scale=100, amplitude=1000,
                              asymmetry=0.9,spread=2)

Obtain residuals before and after distortion correction and plot to inspect, the aim is for any differences to be circularly symmetric

In [ ]:
residuals = cal.get_distortion_residuals(mask_radius=10, spread=2)
residuals.plot(cmap='RdBu', vmax=0.04)

Plot distortion corrected diffraction pattern with adjustable reference circle for inspection

In [ ]:
cal.plot_corrected_diffraction_pattern()

Check the affine matrix, which may be applied to other data

In [ ]:
cal.affine_matrix

# <a href='#cal'></a> 3. Determining Real & Reciprocal Space Scales

Determine the diffraction pattern calibration in reciprocal Angstroms per pixel

In [ ]:
cal.get_diffraction_calibration(mask_length=30,
                                linewidth=5)

Plot the calibrated diffraction data to check it looks about right

In [ ]:
cal.plot_calibrated_data(data_to_plot='au_x_grating_dp',
                         cmap='magma', vmax=0.1)

Plot the cross grating image data to define the line along which to take trace

In [ ]:
line = pxm.roi.Line2DROI(x1=4.83957, y1=44.4148, x2=246.46, y2=119.159, linewidth=5.57199)

cal_lib.plot_calibration_data(data_to_plot='au_x_grating_im',
                              roi=line)

In [ ]:
line

Obtain an orientation map from the matching results, by selecting the top matching result, and plot it.

In [ ]:
trace = line(cal_lib.au_x_grating_im).as_signal1D(0)
trace.plot()

In [ ]:
cal.get_navigation_calibration(line_roi=line, x1=40.,x2=232.,
                               n=3, xspace=500.)